In [9]:
from __future__ import annotations

import random
import socket
import traceback
from datetime import timedelta
from datetime import datetime
from typing import Any, Callable, Literal, Optional

import numpy as np
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import tyro
from pathlib import Path
import os
import yaml

import nerfstudio
from nerfstudio.configs.config_utils import convert_markup_to_ansi
from nerfstudio.configs.method_configs import AnnotatedBaseConfigUnion
from nerfstudio.configs.method_configs import all_methods
from nerfstudio.engine.trainer import TrainerConfig
from nerfstudio.utils import comms, profiler
from nerfstudio.utils.rich_utils import CONSOLE

In [10]:
DEFAULT_TIMEOUT = timedelta(minutes=30)

# speedup for when input size to model doesn't change (much)
torch.backends.cudnn.benchmark = True  # type: ignore

In [11]:
def launch(
    main_func: Callable,
    num_devices_per_machine: int,
    num_machines: int = 1,
    machine_rank: int = 0,
    dist_url: str = "auto",
    config: Optional[TrainerConfig] = None,
    timeout: timedelta = DEFAULT_TIMEOUT,
    device_type: Literal["cpu", "cuda", "mps"] = "cuda",
) -> None:
    """Function that spawns multiple processes to call on main_func

    Args:
        main_func (Callable): function that will be called by the distributed workers
        num_devices_per_machine (int): number of GPUs per machine
        num_machines (int, optional): total number of machines
        machine_rank (int, optional): rank of this machine.
        dist_url (str, optional): url to connect to for distributed jobs.
        config (TrainerConfig, optional): config file specifying training regimen.
        timeout (timedelta, optional): timeout of the distributed workers.
        device_type: type of device to use for training.
    """
    assert config is not None
    world_size = num_machines * num_devices_per_machine
    if world_size == 0:
        raise ValueError("world_size cannot be 0")
    elif world_size == 1:
        # uses one process
        try:
            # print('uses one process') here
            main_func(local_rank=0, world_size=world_size, config=config)
        except KeyboardInterrupt:
            # print the stack trace
            CONSOLE.print(traceback.format_exc())
        finally:
            profiler.flush_profiler(config.logging)

In [12]:
def _set_random_seed(seed) -> None:
    """Set randomness seed in torch and numpy"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


In [13]:
def train_loop(local_rank: int, world_size: int, config: TrainerConfig, global_rank: int = 0):
    """Main training function that sets up and runs the trainer per process

    Args:
        local_rank: current rank of process
        world_size: total number of gpus available
        config: config file specifying training regimen
    """
    _set_random_seed(config.machine.seed + global_rank)
    trainer = config.setup(local_rank=local_rank, world_size=world_size)
    # print('trainer', trainer)
    # <nerfstudio.engine.trainer.Trainer object at 0x7f2cd34ddbe0>
    trainer.setup()
    trainer.train()

In [14]:
config = all_methods['nerfacto']
config.timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S")
config.data = Path('data/nerfstudio/poster')
config.pipeline.datamanager.data = Path('data/nerfstudio/poster')
launch(
        main_func=train_loop,
        num_devices_per_machine=config.machine.num_devices,
        device_type=config.machine.device_type,
        num_machines=config.machine.num_machines,
        machine_rank=config.machine.machine_rank,
        dist_url=config.machine.dist_url,
        config=config,
    )

[12:33:05] Saving checkpoints to: outputs/poster/nerfacto/2023-08-24_123305/nerfstudio_models             ]8;id=234053;file:///home/tianshu/nerfstudio/nerfstudio/engine/trainer.py\trainer.py]8;;\:]8;id=146316;file:///home/tianshu/nerfstudio/nerfstudio/engine/trainer.py#135\135]8;;\

           Auto image downscale factor of 2                                                 ]8;id=91161;file:///home/tianshu/nerfstudio/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=619176;file:///home/tianshu/nerfstudio/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#324\324]8;;\

Setting up training dataset...

Caching all 204 images.

Output()

...

Step (% Done)       Train Iter (time)    ETA (time)           
--------------------------------------------------------------
Step (% Done)       Train Iter (time)    ETA (time)                                                  
--------------------------------------------------------------                                       
0 (0.00%)           2 s, 386.210 ms      19 h, 53 m, 6 s                                             
---------------------------------------------------------------------------------------------------- 
Viewer at: https://viewer.nerf.studio/versions/23-05-15-1/?websocket_url=ws://localhost:7007         
Step (% Done)       Train Iter (time)    ETA (time)           Train Rays / Sec                       
-----------------------------------------------------------------------------------                  
0 (0.00%)           2 s, 386.210 ms      19 h, 53 m, 6 s                                             
10 (0.03%)          262.243 ms           2 h, 11 m, 4 s   

Traceback (most recent call last):
  File "/tmp/ipykernel_22824/4082220732.py", line 31, in launch
    main_func(local_rank=0, world_size=world_size, config=config)
  File "/tmp/ipykernel_22824/35614963.py", line 14, in train_loop
    trainer.train()
  File "/home/tianshu/nerfstudio/nerfstudio/engine/trainer.py", line 297, in train
    writer.write_out_storage()
  File "/home/tianshu/nerfstudio/nerfstudio/utils/decorators.py", line 82, in wrapper
    ret = func(*args, **kwargs)
  File "/home/tianshu/nerfstudio/nerfstudio/utils/writer.py", line 165, in write_out_storage
    writer.write_stats_log(EVENT_STORAGE[0]["step"])
  File "/home/tianshu/nerfstudio/nerfstudio/utils/decorators.py", line 82, in wrapper
    ret = func(*args, **kwargs)
  File "/home/tianshu/nerfstudio/nerfstudio/utils/writer.py", line 415, in write_stats_log
    self._print_stats(latest_map)
  File "/home/tianshu/nerfstudio/nerfstudio/utils/decorators.py", line 82, in wrapper
    ret = func(*args, **kwargs)
  File "/home/tianshu/nerfstudio/nerfstudio/utils/writer.py", line 487, in _print_stats
    _cursorup(cursor_idx)
  File "/home/tianshu/nerfstudio/nerfstudio/utils/writer.py", line 355, in _cursorup
    print(f"\r\033[{x}A", end="\x1b[1K\r")
  File "/home/tianshu/anaconda3/envs/nerfstudio/lib/python3.8/site-packages/ipykernel/iostream.py", line 648, in write
    self._schedule_flush()
  File "/home/tianshu/anaconda3/envs/nerfstudio/lib/python3.8/site-packages/ipykernel/iostream.py", line 545, in 
_schedule_flush
    self.pub_thread.schedule(_schedule_in_thread)
  File "/home/tianshu/anaconda3/envs/nerfstudio/lib/python3.8/site-packages/ipykernel/iostream.py", line 251, in 
schedule
    self._event_pipe.send(b"")
  File "/home/tianshu/anaconda3/envs/nerfstudio/lib/python3.8/site-packages/ipykernel/iostream.py", line 131, in 
_event_pipe
    event_pipe = self._local.event_pipe
KeyboardInterrupt

Printing profiling stats, from longest to shortest duration in seconds

Trainer.train_iteration: 0.0457

VanillaPipeline.get_train_loss_dict: 0.0251